<a href="https://colab.research.google.com/github/leekate/titanic_pro/blob/master/Titanic02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle: Titanic_Machine Learning from Disaster
#### 목표:트레이닝 세트에서 패턴을 찾고 살아남은 사람을 예측하기

In [ ]:
#데이터 확인에 필요한 라이브러리 불러오기
import numpy as np 
import pandas as pd 

# 1. 데이터 확인부터 하자

In [ ]:
train_data = pd.read_csv("train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("test.csv")
test_data.head()

항목별 의미이다.
*   Survived: 0 = 아니오, 1 = 예
*   pclass: 객실 클래스 / 1 = 1등석, 2 = 2등석, 3 = 3등석 
*   Name : 탑승자 이름
*   Sex : 탑승자 성별
*   Age : 탑승자 나이
*   pclass: 티켓 클래스 / 1 = 1위, 2 = 2위, 3 = 3위
*   SibSp: 타이타닉호에 탑승한 형제/배우자 수
*   parch: 타이타닉호에 승선한 부모/아이들의 수
*   TIcket: 티켓 번호
*   Fare : 여객운임
*   Cabin: 객실 번호
*   Embarked: 탑승 항만 / C = 체르부르, Q = 퀸스타운, S = 사우샘프턴

## 간단하게 bar로 표현
몇등석에 있었는지에 따라 살아남은 정도가 궁금해졌다. 좋은 1등석에 있을수록 많은 사람이 살아남았을까?

In [ ]:
import matplotlib.pyplot as plt

train_data_class = train_data.groupby(['Pclass'])['Survived']
train_data_class.plot.bar()

그래프를 알아보기 힘들정도로 복잡하다. 따라서 다른 방법으로 보기 쉽게 조절해주어야 한다. mean() 메소드를 사용해보자.

In [ ]:
import matplotlib.pyplot as plt

train_data_class = train_data.groupby(['Pclass'])['Survived'].mean()
train_data_class.plot.bar()

알고싶었던 정보를 쉽게 알 수 있다. 성별과도 비교해보자.

In [ ]:
import matplotlib.pyplot as plt

train_data_sex = train_data.groupby(['Sex'])['Survived'].mean()
train_data_sex.plot.bar()


In [ ]:
import matplotlib.pyplot as plt

train_data_sex = train_data.groupby(['Age'])['Survived'].mean()
train_data_sex.plot.bar()

나이가 너무 다양해서 그래프가 너무 복잡하다. 이건 아래서 해결해보자.

## 더 멋있는 바 차트를 만들어보자

In [ ]:
def better_bar(feature):
  yes=train_data[train_data['Survived']==1][feature].value_counts()
  no=train_data[train_data['Survived']==0][feature].value_counts()
  df=pd.DataFrame([yes,no])
  df.index=['yes','no']
  df.plot(kiind='bar',stacked=True,figsize=(10,5))

In [ ]:
better_bar("Parch")

# 왜 안되는걸까!!!!!!!!!!!!

앗 그런데 pandas를 확인했을 때 비어보이는 값들이 많다. missing data를 파악해보자

In [ ]:
test_data.info()

여기까지 확인했을 때 cabin의 값을 가지는 샘플이 많지 않다는 것을 알 수 있다. 샘플이 부족하면 언더피팅이 발생할 수 있어 샘플을 채워주거나 모델을 복잡하게 하거나 규제를 줄여 언더피팅 발생을 막아야 한다. 하지만 cabin이라는 항목이 값을 채워주기에는 너무 많은 양이 존재하지 않고, 살아남은 사람을 예상하는 것과 큰 연관이 없다고 생각해 이 데이터를 빼고 생각하려고 한다.

In [ ]:

train_new=train_data.drop('Cabin',axis=1)

train_new

성별이 문자형이므로 다루기 쉽게 숫자형으로 바꾸어주자

In [ ]:

train_new.Sex = train_data.Sex.replace('female',0)
train_new.Sex = train_data.Sex.replace('male',1)

train_new

단일 특성을 가지고 생존 확률을 예측하기보단 여러가지 특성을 종합해서 예측을 하는 것이 더 좋을 것이다.

# 2. 데이터 전처리를 해보자

### Embarked 파트

In [ ]:
train_new.Embarked.value_counts(dropna=False)

위의 코드는 dropna를 false로 했기 떄문에 nan값을 포함한 갯수의 총합을 의미한다. s가 많기 때문에 누락된 값을 s로 채워주자. 

In [ ]:

train_new.Embarked = train_data.Embarked.fillna('S')

train_new


### Age 파트

age 전처리도 해보자

위에서 봤던 그래프를 기억하는가? 나이가 너무 다양해서 한눈에 파악하기 쉽지 않았다. 카테고리를 만들어서 나이의 구역을 나누어 그래프로 표현한다면 더 보기 쉬울 거 같다.해보자.

In [ ]:
for dataset in train_new:
​    dataset['Age'].fillna(dataset['Age'].mean(), inplace=True)
​    dataset['Age'] = dataset['Age'].astype(int)
​    train_new['AgeBand'] = pd.cut(train_new['Age'], 5)
print (train_new[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean()) 

# 왜 안되는 거지!!!!!!!!!

In [ ]:
for dataset in train_and_test:
​    dataset.loc[ dataset['Age'] <= 19, 'Age'] = 0
​    dataset.loc[(dataset['Age'] > 19) & (dataset['Age'] <= 38), 'Age'] = 1
​    dataset.loc[(dataset['Age'] > 38) & (dataset['Age'] <= 57), 'Age'] = 2
​    dataset.loc[(dataset['Age'] > 57) & (dataset['Age'] <= 76), 'Age'] = 3
​    dataset.loc[ dataset['Age'] > 76, 'Age'] = 4
​    dataset['Age'] = dataset['Age'].map( { 0: 'Child',  1: 'Young', 2: 'Middle', 3: 'Next_age', 4: 'Old'} ).astype(str)

위에가 안됐으니 이게 당연히 안되지

### Fare 파트

### 나머지 파트


# 모델링 및 학습

# 결정트리로도 해보자